In [6]:
import sqlite3
from contextlib import contextmanager
from sqlite3 import Error

database = "./ds_hw_02.db"

@contextmanager
def create_connection(db_file):   #creates connection to SQLite db file
    conn = sqlite3.connect(db_file)    
    yield conn
    conn.rollback()
    conn.close()


In [28]:
def make_table_from_list(header_list:list,query_result:list)->str:
    # Find the maximum width for each column
    max_column_widths = [min(40, max(len(str(name)), max(len(str(value)) for value in column))) for name, column in zip(header_list, zip(*query_result))]

    # Print table header
    header = '| ' + ' | '.join(f'{name[:37]:<{width}}' + '...' if len(name) > 40 else f'{name:<{width}}' for name, width in zip(header_list, max_column_widths)) + ' |'
    line = '-' * len(header)
    message = f'{line}\n{header}\n{line}\n'

    # Print rows
    for row in query_result:
        row_str = '| ' + ' | '.join(f'{str(value)[:37]:<{width}}' + '...' if len(str(value)) > 40 else f'{str(value):<{width}}' for value, width in zip(row, max_column_widths)) + ' |'
        message += f'{row_str}\n'
    return message

In [8]:
with create_connection(database) as conn:                              #gets all tasks
    #Отримати всі завдання певного користувача. Використайте SELECT для отримання завдань конкретного користувача за його user_id.
    sql = "SELECT * FROM tasks WHERE user_id = ?;"
    cur = conn.cursor()
    cur.execute(sql, (1,))            
    print(cur.fetchall())

[(11, 1, 1, 'Create New Product gather information.', 'Quite guess structure drop.\nNews box rock billion. Create move opportunity administration young language. Responsibility impact experience summer pretty.'), (12, 3, 1, 'In This Month in 3 days.', 'Single full write near above. Somebody collection ground poor be. Authority writer price wind or daughter. Seem second you though example stage war cup.'), (13, 1, 1, 'Create New Product ToDo task.', 'Business training leader current image push form. Pay language full reveal always analysis. Author finally book concern.'), (14, 1, 1, 'Minimize Costs gather information.', ''), (15, 3, 1, 'Finish Job finish project.', 'Material ahead business generation. Main great real always bag draw boy matter. Participant night truth if practice.'), (16, 3, 1, 'Todo Task finish project.', 'Event beyond much management too. Live personal from debate sound. Service very once start traditional.\nTable another any me thought strong summer. Seem mouth on in

In [15]:
#Вибрати завдання за певним статусом. Використайте підзапит для вибору завдань з конкретним статусом, наприклад, 'new'.
# Отримати всі завдання, які ще не завершено. Виберіть завдання, чий статус не є 'завершено'.
with create_connection(database) as conn:                              
    notin = False
    status = "new"
    try:
        if notin:
            sql = "SELECT t.title, s.name FROM tasks t JOIN status s ON t.status_id = s.id WHERE NOT s.name = ?;"
        else:
            sql = "SELECT t.title, s.name FROM tasks t JOIN status s ON t.status_id = s.id WHERE s.name = ?;"
        cur = conn.cursor()
        cur.execute(sql, (status,))            
        results = cur.fetchall()
        message = ""
        for result in results:
            print(result)
    except Error as e:
        print(f"Error in queries.py in get_tasks_qty_by_statuses: {e}")
    finally:
        cur.close()
    


('Achieve A Mission minimize costs.', 'new')
('Minimize Costs till the EOW.', 'new')
("At Monday 9 O'Clock at monday 9 o'clock.", 'new')
('Create New Product gather information.', 'new')
('Create New Product ToDo task.', 'new')
('Minimize Costs gather information.', 'new')
('Todo Task in 3 days.', 'new')
('Get The Result finish project.', 'new')
('Todo Task achieve a mission.', 'new')
('In This Month gather information.', 'new')
('Do morning routine', 'new')


In [16]:
# Отримати кількість завдань для кожного статусу. Використайте SELECT, COUNT, GROUP BY для групування завдань за статусами.
with create_connection(database) as conn:                         
    sql = "SELECT s.name, COUNT(t.id) FROM tasks t JOIN status s ON t.status_id = s.id GROUP BY s.name;"
    cur = conn.cursor()
    cur.execute(sql)            
    print(cur.fetchall())

[('completed', 9), ('in progress', 10), ('new', 11)]


In [26]:
# Отримати список завдань, що не мають опису. Виберіть завдання, у яких відсутній опис.
with create_connection(database) as conn:                         
    sql = "SELECT t.title, t.description FROM tasks t WHERE t.description = '';"
    cur = conn.cursor()
    cur.execute(sql)            
    print(cur.fetchall())

[("At Monday 9 O'Clock at monday 9 o'clock.", ''), ('In This Month till the EOW.', '')]


In [18]:
# Вибрати користувачів та їхні завдання, які є у статусі 'in progress'. 
#          Використайте INNER JOIN для отримання списку користувачів та їхніх завдань із певним статусом.
with create_connection(database) as conn:                         
    sql = "SELECT s.name, t.title, u.fullname FROM tasks t JOIN status s ON t.status_id = s.id JOIN users u ON t.user_id = u.id WHERE s.name = ?;"
    cur = conn.cursor()
    cur.execute(sql, ("in progress",))            
    print(cur.fetchall())

[('in progress', 'Todo Task in this month.', 'Anthony Jones'), ('in progress', 'Create New Product gather information.', 'Anthony Jones'), ('in progress', 'Minimize Costs in 3 days.', 'Anthony Jones'), ('in progress', "At Monday 9 O'Clock minimize costs.", 'Anthony Jones'), ('in progress', "Minimize Costs at monday 9 o'clock.", 'William Adams'), ('in progress', 'Achieve A Mission minimize costs.', 'William Adams'), ('in progress', 'In 2 Hours achieve a mission.', 'Julie Blake'), ('in progress', 'In This Month achieve a mission.', 'Julie Blake'), ('in progress', 'Create New Product minimize costs.', 'Julie Blake'), ('in progress', 'Finish Project gather information.', 'Julie Blake')]


In [30]:
# Отримати користувачів та кількість їхніх завдань. 
#          Використайте LEFT JOIN та GROUP BY для вибору користувачів та підрахунку їхніх завдань.
# Отримати список користувачів, які не мають жодного завдання. Використайте комбінацію SELECT, WHERE NOT IN і підзапит.

with create_connection(database) as conn:                         
    no_task = False
    try:
        cur = conn.cursor()
        if no_task:
            sql = "SELECT u.fullname FROM users u WHERE u.id NOT IN (SELECT t.user_id FROM tasks t GROUP BY t.user_id)"
        else:
            sql = "SELECT u.fullname,COUNT(t.id) FROM users u LEFT JOIN tasks t ON t.user_id = u.id GROUP BY u.fullname;"
        cur.execute(sql)            
        results = cur.fetchall()
        description_list = [description[0] for description in cur.description]
        message = make_table_from_list(description_list,results)
    except Error as e:
        message = f"Error in queries.py in get_users_by_tasks: {e}"
    finally:
        cur.close()
    print(message)

--------------------------------
| fullname       | COUNT(t.id) |
--------------------------------
| Alan Ellis     | 10          |
| Brian Lawrence | 9           |
| QQQ            | 0           |
| QQWW           | 10          |



In [20]:
# Отримати список користувачів, які не мають жодного завдання. Використайте комбінацію SELECT, WHERE NOT IN і підзапит.
with create_connection(database) as conn:                         
    sql = "SELECT u.fullname FROM users u WHERE u.id NOT IN (SELECT t.user_id FROM tasks t GROUP BY t.user_id)"
    cur = conn.cursor()
    cur.execute(sql)            
    print(cur.fetchall())

[]


In [32]:
# Отримати завдання, які призначені користувачам з певною доменною частиною електронної пошти. 
#          Використайте SELECT з умовою LIKE в поєднанні з JOIN, щоб вибрати завдання, призначені користувачам, 
#          чия електронна пошта містить певний домен (наприклад, '%@example.com').
with create_connection(database) as conn:                         
    email_like = '%@example.com'
    try:   
        sql = "SELECT u.fullname,u.email,t.title FROM tasks t JOIN users u ON t.user_id = u.id WHERE u.email LIKE ?;"
        cur = conn.cursor()
        cur.execute(sql,(email_like,))            
        results = cur.fetchall()
        description_list = [description[0] for description in cur.description]
        message = make_table_from_list(description_list,results)
    except Error as e:
        message = f"Error in queries.py in get_users_by_mail: {e}"
    finally:
        cur.close()
    print(message)

------------------------------------------------------------------------------------------
| fullname   | email                          | title                                    |
------------------------------------------------------------------------------------------
| Alan Ellis | christophermorales@example.com | Create New Product till the EOW.         |
| Alan Ellis | christophermorales@example.com | Achieve A Mission till the EOW.          |
| Alan Ellis | christophermorales@example.com | Finish Project in this month.            |
| Alan Ellis | christophermorales@example.com | Finish Job get the result.               |
| Alan Ellis | christophermorales@example.com | Till The Eow in 2 hours.                 |
| Alan Ellis | christophermorales@example.com | Achieve A Mission till the EOW.          |
| Alan Ellis | christophermorales@example.com | Create New Product create new product.   |
| Alan Ellis | christophermorales@example.com | In 2 Hours gather information.           |

In [ ]:
# Оновити статус конкретного завдання. Змініть статус конкретного завдання на 'in progress' або інший статус.
with create_connection(database) as conn:                         
    sql = "UPDATE tasks SET status_id = (select s.id FROM status s WHERE s.name =?) WHERE id = ?;"
    cur = conn.cursor()
    cur.execute(sql,("new",10))
    conn.commit()
    sql = "Select * FROM tasks WHERE id = ?;"
    cur.execute(sql,(10,))
    print(cur.fetchall())

In [ ]:
# Додати нове завдання для конкретного користувача. Використайте INSERT для додавання нового завдання.
with create_connection(database) as conn:                         
    sql = """INSERT INTO tasks (status_id,user_id,title,description) VALUES(?,?,?,?);
    """
    cur = conn.cursor()
    cur.execute(sql,(1,4,"Do morning routine", "every day"))
    conn.commit()
    
    sql = "Select * FROM tasks WHERE user_id = ?;"
    cur.execute(sql,(4,))
    print(cur.fetchall())

In [ ]:
# Видалити конкретне завдання. Використайте DELETE для видалення завдання за його id.
with create_connection(database) as conn:                         
    cur = conn.cursor()
    sql = "Select * FROM tasks WHERE id = ?;"
    cur.execute(sql,(24,))
    print(cur.fetchall())

    sql = """DELETE FROM tasks WHERE id = ?;
    """
    cur = conn.cursor()
    cur.execute(sql,(24,))
    conn.commit()

    sql = "Select * FROM tasks WHERE id = ?;"
    cur.execute(sql,(24,))
    print(cur.fetchall())

In [ ]:
# Оновити ім'я користувача. Змініть ім'я користувача за допомогою UPDATE.
with create_connection(database) as conn:                         
    sql = "UPDATE users SET fullname = ? WHERE id = ?;"
    cur = conn.cursor()
    cur.execute(sql,("Don Jow",4))
    conn.commit()
    sql = "Select * FROM users WHERE id =?"
    cur.execute(sql,(4,))
    print(cur.fetchall())

In [25]:
print(bool(""))

False
